In [1]:
!pip install neo4j

In [43]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
from ast import literal_eval
import json

In [3]:
from ast import literal_eval
from tabulate import tabulate
import json
from z3 import *
import pandas as pd
import numpy as np

In [45]:
# Database Credentials

uri             = "bolt://localhost:7687"

userName        = "neo4j"

password        = "nlp_sem3"

# Connect to the neo4j database server
graphDB_Driver  = GraphDatabase.driver(uri, auth=(userName, password))

In [46]:
#Load the Crossword to solve
with open('mini_nyt_dec16.json', 'r') as f:
    crossword_to_solve = f.read()
crossword_to_solve = literal_eval(str(crossword_to_solve))
crossword_to_solve

{'Milk option': {'start': (0, 1),
  'direction': 'A',
  'length': 4,
  'answer': 'SKIM'},
 '5x5 crossword puzzle': {'start': (1, 1),
  'direction': 'A',
  'length': 4,
  'answer': 'MINI'},
 '___ Monsoon, drag queen who has won two seasons of "RuPauls Drag Race"': {'start': (2,
   0),
  'direction': 'A',
  'length': 5,
  'answer': 'JINKX'},
 'Word in some shoppe names': {'start': (3, 0),
  'direction': 'A',
  'length': 4,
  'answer': 'OLDE'},
 'Bit on a friendship bracele': {'start': (4, 0),
  'direction': 'A',
  'length': 4,
  'answer': 'BEAD'},
 '"The ) of :-)"': {'start': (0, 1),
  'direction': 'D',
  'length': 5,
  'answer': 'SMILE'},
 'Sorta': {'start': (0, 2), 'direction': 'D', 'length': 5, 'answer': 'KINDA'},
 'Signed, as a contract': {'start': (0, 3),
  'direction': 'D',
  'length': 5,
  'answer': 'INKED'},
 'A little bit of this, a little bit of that': {'start': (0, 4),
  'direction': 'D',
  'length': 3,
  'answer': 'MIX'},
 'What pays the bills': {'start': (2, 0),
  'direction

In [53]:
# Visualize the extracted Crossword data

puzzle = []
for key in crossword_to_solve.keys():
    puzzle.append((key, crossword_to_solve[key]['start'], crossword_to_solve[key]['direction'], crossword_to_solve[key]['answer']))

puzzle_pd = pd.DataFrame(puzzle, columns=['clue','RC','direction','answer'])
puzzle_pd

,clue,RC,direction,answer
0,Milk option,"(0, 1)",A,SKIM
1,5x5 crossword puzzle,"(1, 1)",A,MINI
2,"___ Monsoon, drag queen who has won two season...","(2, 0)",A,JINKX
3,Word in some shoppe names,"(3, 0)",A,OLDE
4,Bit on a friendship bracele,"(4, 0)",A,BEAD
5,"""The ) of :-)""","(0, 1)",D,SMILE
6,Sorta,"(0, 2)",D,KINDA
7,"Signed, as a contract","(0, 3)",D,INKED
8,"A little bit of this, a little bit of that","(0, 4)",D,MIX
9,What pays the bills,"(2, 0)",D,JOB


In [50]:
crossword_to_solve['Milk option']['start']

(0, 1)

#### For demo we can add the correct answer here to show that solver does work if given correct answer

In [26]:
crossword_hints = pd.read_csv("miniIanswers.csv")
crossword_hints = crossword_hints.drop(columns=['Unnamed: 0'], axis=1)
crossword_hints.head()
mini_hints_clues = {}
i = 0
max = 10
m = 0
master = {}
for key in crossword_to_solve.keys():
    clues = np.array(crossword_hints.iloc[i:i+1,:])  
    master = {}
    m = 0
    for k in range(len(clues[0])):
        if clues[0][k] not in master and len(clues[0][k]) == crossword_to_solve[key]['length']:
             master[clues[0][k]] = k
             m = m + 1
        if m == 10:
            break
    
    master[crossword_to_solve[key]['answer']] =  0 ## hack to validate crossword by injecting true answer
    mini_hints_clues[key] = master
    i = i + 1
    
mini_hints_clues  
with open("mini_nyt_dec16_hint.json", "w") as outfile:
    json.dump(literal_eval(str(mini_hints_clues)), outfile) 

In [33]:
translation_table = dict.fromkeys(map(ord, "!@#$'"), None)
# line = "IT'S"
# line.translate(translation_table)

In [54]:
#Load the hints to solve

with open("mini_nyt_dec16_hint_raw.json", 'r') as f:
    crossword_hints = f.read()
crossword_hints = literal_eval(str(crossword_hints))
crossword_hints

{'Milk option': {'AABB': 109,
  'ESCS': 224,
  'ALYN': 431,
  'TMNT': 436,
  'USIA': 500,
  'IVAR': 510,
  'BCDE': 518,
  'CMAS': 526,
  'SEVE': 527,
  'CERN': 597,
  'INCE': 924,
  'FINA': 940,
  'GSIX': 954,
  'WWII': 957,
  'ESOP': 978},
 '5x5 crossword puzzle': {'RAFA': 12,
  'LINC': 49,
  'TAMI': 63,
  'BAYH': 189,
  'ALYN': 196,
  'HERC': 239,
  'MOSE': 277,
  'FALK': 287,
  'RULA': 304,
  'ARYA': 307,
  'CAIT': 314,
  'RIPA': 364,
  'HODA': 388,
  'MASA': 429,
  'BROM': 432,
  'TALA': 435,
  'FENN': 436,
  'JAYZ': 471,
  'RAFE': 680,
  'IVER': 762,
  'ILER': 763,
  'KASI': 782,
  'VERE': 791,
  'BOLE': 822,
  'LEGG': 847,
  'DODI': 924,
  'LEEJ': 935,
  'CRIS': 941,
  'IVEY': 946,
  'RALF': 955,
  'ONZE': 988},
 '___ Monsoon, drag queen who has won two seasons of "RuPauls Drag Race"': {'LOHAN': 0,
  'ARKIN': 85,
  'CROCE': 109,
  'TRACI': 129,
  'JAFFE': 153,
  'HEDER': 188,
  'ALANA': 192,
  'IRVIN': 223,
  'JONZE': 241,
  'LIDDY': 313,
  'ROARK': 318,
  'TRUDI': 330,
  'VERNA'

In [42]:
#Store the clues in Graph DB

clue_dic = {""}
error_dic = {""}
with graphDB_Driver.session() as graphDB_Session:
    c=1
    a=1
    for clue in crossword_to_solve:
        
        if not clue in clue_dic:
            
            i=1
            cmd = ""
            try:
                ans = crossword_to_solve[clue]['answer']
#                 cmd = "MERGE (c" + str(c) + ":CLUE{value: '" + clue +"'}) " + \
                cmd = "MERGE (c" + str(c) + ":CLUE{value: '" + clue +"', length:" + str(len(ans))+"}) " + \
                      "MERGE (a" + str(a) + ":TRUEANSWER{value: '" + ans + "', length: " + str(len(ans))+"}) "
                
                for char in ans:
                    if char.isalpha():
                        idx = char+str(i)
                        cmd += "MERGE (" + idx + ":LETTER{value:'"+ char + "', index:" + str(i) + "}) "
                        cmd += "MERGE (a" + str(a) + ")-[:LETTERS]->(" + idx + ")  "
                       
                        i = i + 1
                cmd += "MERGE (c" + str(c) + ")-[:ANSWERS]->(a" + str(a) + ")  "
                a=a+1
                
            except Exception as e: 
                print(e)
                print("ans is:", ans)
                error_dic.add(ans)
            try:
#                 graphDB_Session.run(cmd)
                print(cmd) 
                print("\n")
            except Exception as e: 
                print(e)
                print("ans is:", ans)
                error_dic.add(ans)
            c = c + 1
            clue_dic.add(ans)

MERGE (c1:CLUE{value: 'Milk option', length:4}) MERGE (a1:TRUEANSWER{value: 'SKIM', length: 4}) MERGE (S1:LETTER{value:'S', index:1}) MERGE (a1)-[:LETTERS]->(S1)  MERGE (K2:LETTER{value:'K', index:2}) MERGE (a1)-[:LETTERS]->(K2)  MERGE (I3:LETTER{value:'I', index:3}) MERGE (a1)-[:LETTERS]->(I3)  MERGE (M4:LETTER{value:'M', index:4}) MERGE (a1)-[:LETTERS]->(M4)  MERGE (c1)-[:ANSWERS]->(a1)  


MERGE (c2:CLUE{value: '5x5 crossword puzzle', length:4}) MERGE (a2:TRUEANSWER{value: 'MINI', length: 4}) MERGE (M1:LETTER{value:'M', index:1}) MERGE (a2)-[:LETTERS]->(M1)  MERGE (I2:LETTER{value:'I', index:2}) MERGE (a2)-[:LETTERS]->(I2)  MERGE (N3:LETTER{value:'N', index:3}) MERGE (a2)-[:LETTERS]->(N3)  MERGE (I4:LETTER{value:'I', index:4}) MERGE (a2)-[:LETTERS]->(I4)  MERGE (c2)-[:ANSWERS]->(a2)  


MERGE (c3:CLUE{value: '___ Monsoon, drag queen who has won two seasons of "RuPauls Drag Race"', length:5}) MERGE (a3:TRUEANSWER{value: 'JINKX', length: 5}) MERGE (J1:LETTER{value:'J', index:1}) MERGE

In [41]:
#Store the hints in the Graph Db

clue_dic = {""}
error_dic = {""}
with graphDB_Driver.session() as graphDB_Session:
    for clue in crossword_hints:
        
        if not clue in clue_dic:
            i=1
            cmd = ""
            for ans in crossword_hints[clue]:
                rank = crossword_hints[clue][ans]
                try:
                    
                    cmd = "MATCH (c1:CLUE{value: '" + clue +"'}) " + \
                          "MERGE (a1:ANSWER{value: '" + ans.translate(translation_table) + "', rank: " + str(rank) + ", length: " + str(len(ans))+"}) "
                    i=1
                    for char in ans:
                        if char.isalpha():
                            idx = char+str(i)
                            cmd += "MERGE (" + idx + ":LETTER{value:'"+ char + "', index:" + str(i) + "}) "
                            cmd += "MERGE (a1)-[:LETTERS]->(" + idx + ")  "
                            cmd += "MERGE (c1)-[:ANSWERS]->(a1)  "
                            i = i + 1
                except Exception as e: 
                    print(e)
                    print("ans is:", ans)
                    error_dic.add(ans)
                try:
                    graphDB_Session.run(cmd)
                    print(cmd) 
                    print("\n")
                except Exception as e: 
                    print(e)
                    print("ans is:", ans)
                    error_dic.add(ans)
            clue_dic.add(ans)

MATCH (c1:CLUE{value: 'Milk option'}) MERGE (a1:ANSWER{value: 'AABB', rank: 109, length: 4}) MERGE (A1:LETTER{value:'A', index:1}) MERGE (a1)-[:LETTERS]->(A1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A2:LETTER{value:'A', index:2}) MERGE (a1)-[:LETTERS]->(A2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (B3:LETTER{value:'B', index:3}) MERGE (a1)-[:LETTERS]->(B3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (B4:LETTER{value:'B', index:4}) MERGE (a1)-[:LETTERS]->(B4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'Milk option'}) MERGE (a1:ANSWER{value: 'ESCS', rank: 224, length: 4}) MERGE (E1:LETTER{value:'E', index:1}) MERGE (a1)-[:LETTERS]->(E1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (S2:LETTER{value:'S', index:2}) MERGE (a1)-[:LETTERS]->(S2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (C3:LETTER{value:'C', index:3}) MERGE (a1)-[:LETTERS]->(C3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (S4:LETTER{value:'S', index:4}) MERGE (a1)-[:LETTERS]->(S4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'Milk option'}) ME

MATCH (c1:CLUE{value: '5x5 crossword puzzle'}) MERGE (a1:ANSWER{value: 'RULA', rank: 304, length: 4}) MERGE (R1:LETTER{value:'R', index:1}) MERGE (a1)-[:LETTERS]->(R1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (U2:LETTER{value:'U', index:2}) MERGE (a1)-[:LETTERS]->(U2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (L3:LETTER{value:'L', index:3}) MERGE (a1)-[:LETTERS]->(L3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A4:LETTER{value:'A', index:4}) MERGE (a1)-[:LETTERS]->(A4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: '5x5 crossword puzzle'}) MERGE (a1:ANSWER{value: 'ARYA', rank: 307, length: 4}) MERGE (A1:LETTER{value:'A', index:1}) MERGE (a1)-[:LETTERS]->(A1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (R2:LETTER{value:'R', index:2}) MERGE (a1)-[:LETTERS]->(R2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (Y3:LETTER{value:'Y', index:3}) MERGE (a1)-[:LETTERS]->(Y3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A4:LETTER{value:'A', index:4}) MERGE (a1)-[:LETTERS]->(A4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 

MATCH (c1:CLUE{value: 'Word in some shoppe names'}) MERGE (a1:ANSWER{value: 'ATOI', rank: 29, length: 4}) MERGE (A1:LETTER{value:'A', index:1}) MERGE (a1)-[:LETTERS]->(A1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (T2:LETTER{value:'T', index:2}) MERGE (a1)-[:LETTERS]->(T2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (O3:LETTER{value:'O', index:3}) MERGE (a1)-[:LETTERS]->(O3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (I4:LETTER{value:'I', index:4}) MERGE (a1)-[:LETTERS]->(I4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'Word in some shoppe names'}) MERGE (a1:ANSWER{value: 'AHEN', rank: 78, length: 4}) MERGE (A1:LETTER{value:'A', index:1}) MERGE (a1)-[:LETTERS]->(A1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (H2:LETTER{value:'H', index:2}) MERGE (a1)-[:LETTERS]->(H2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (E3:LETTER{value:'E', index:3}) MERGE (a1)-[:LETTERS]->(E3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (N4:LETTER{value:'N', index:4}) MERGE (a1)-[:LETTERS]->(N4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE

MATCH (c1:CLUE{value: 'Bit on a friendship bracele'}) MERGE (a1:ANSWER{value: 'SHEL', rank: 359, length: 4}) MERGE (S1:LETTER{value:'S', index:1}) MERGE (a1)-[:LETTERS]->(S1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (H2:LETTER{value:'H', index:2}) MERGE (a1)-[:LETTERS]->(H2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (E3:LETTER{value:'E', index:3}) MERGE (a1)-[:LETTERS]->(E3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (L4:LETTER{value:'L', index:4}) MERGE (a1)-[:LETTERS]->(L4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'Bit on a friendship bracele'}) MERGE (a1:ANSWER{value: 'RALF', rank: 418, length: 4}) MERGE (R1:LETTER{value:'R', index:1}) MERGE (a1)-[:LETTERS]->(R1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A2:LETTER{value:'A', index:2}) MERGE (a1)-[:LETTERS]->(A2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (L3:LETTER{value:'L', index:3}) MERGE (a1)-[:LETTERS]->(L3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (F4:LETTER{value:'F', index:4}) MERGE (a1)-[:LETTERS]->(F4)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c

MATCH (c1:CLUE{value: 'Sorta'}) MERGE (a1:ANSWER{value: 'NUTRA', rank: 62, length: 5}) MERGE (N1:LETTER{value:'N', index:1}) MERGE (a1)-[:LETTERS]->(N1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (U2:LETTER{value:'U', index:2}) MERGE (a1)-[:LETTERS]->(U2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (T3:LETTER{value:'T', index:3}) MERGE (a1)-[:LETTERS]->(T3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (R4:LETTER{value:'R', index:4}) MERGE (a1)-[:LETTERS]->(R4)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A5:LETTER{value:'A', index:5}) MERGE (a1)-[:LETTERS]->(A5)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'Sorta'}) MERGE (a1:ANSWER{value: 'PARKE', rank: 64, length: 5}) MERGE (P1:LETTER{value:'P', index:1}) MERGE (a1)-[:LETTERS]->(P1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A2:LETTER{value:'A', index:2}) MERGE (a1)-[:LETTERS]->(A2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (R3:LETTER{value:'R', index:3}) MERGE (a1)-[:LETTERS]->(R3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (K4:LETTER{value:'K', index:4}) MERGE (a1)-[:LETT

MATCH (c1:CLUE{value: 'Signed, as a contract'}) MERGE (a1:ANSWER{value: 'ORURO', rank: 84, length: 5}) MERGE (O1:LETTER{value:'O', index:1}) MERGE (a1)-[:LETTERS]->(O1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (R2:LETTER{value:'R', index:2}) MERGE (a1)-[:LETTERS]->(R2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (U3:LETTER{value:'U', index:3}) MERGE (a1)-[:LETTERS]->(U3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (R4:LETTER{value:'R', index:4}) MERGE (a1)-[:LETTERS]->(R4)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (O5:LETTER{value:'O', index:5}) MERGE (a1)-[:LETTERS]->(O5)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'Signed, as a contract'}) MERGE (a1:ANSWER{value: 'CISSY', rank: 86, length: 5}) MERGE (C1:LETTER{value:'C', index:1}) MERGE (a1)-[:LETTERS]->(C1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (I2:LETTER{value:'I', index:2}) MERGE (a1)-[:LETTERS]->(I2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (S3:LETTER{value:'S', index:3}) MERGE (a1)-[:LETTERS]->(S3)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (S4:LETTER{value:

MATCH (c1:CLUE{value: 'What pays the bills'}) MERGE (a1:ANSWER{value: 'OO7', rank: 229, length: 3}) MERGE (O1:LETTER{value:'O', index:1}) MERGE (a1)-[:LETTERS]->(O1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (O2:LETTER{value:'O', index:2}) MERGE (a1)-[:LETTERS]->(O2)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'What pays the bills'}) MERGE (a1:ANSWER{value: 'RSA', rank: 230, length: 3}) MERGE (R1:LETTER{value:'R', index:1}) MERGE (a1)-[:LETTERS]->(R1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (S2:LETTER{value:'S', index:2}) MERGE (a1)-[:LETTERS]->(S2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (A3:LETTER{value:'A', index:3}) MERGE (a1)-[:LETTERS]->(A3)  MERGE (c1)-[:ANSWERS]->(a1)  


MATCH (c1:CLUE{value: 'What pays the bills'}) MERGE (a1:ANSWER{value: 'JOB', rank: 0, length: 3}) MERGE (J1:LETTER{value:'J', index:1}) MERGE (a1)-[:LETTERS]->(J1)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (O2:LETTER{value:'O', index:2}) MERGE (a1)-[:LETTERS]->(O2)  MERGE (c1)-[:ANSWERS]->(a1)  MERGE (B3:LETTER{value:'

In [55]:
GRID = crossword_to_solve
GRID_CLUES = crossword_hints

In [63]:
class Crossword():
    def __init__(self):
        pass

    def apply_constraints(self):
        z3_solver = z3.Solver()
        z3_solver.add(self.apply_faiss_answer_constraint())
        z3_solver.add(self.apply_RC_constraint())
        chk = z3_solver.check()
        if str(chk) == 'unsat': #Unstatisfied
            print("Unsolved: Provided answers do not satisfy the Crossword Puzzle.")
            return None
        else:
            print("Solved: Provided answers satisfy the Crossword Puzzle.")
            return z3_solver.model() #Satisfied
        return None
    
    def apply_faiss_answer_constraint(self):
        clues = self.convert_clues_code()
        matrix, start_positions, max_val = self.get_grid()

        clue_constraint = list()

        for x_index in range(max_val):
            for y_index in range(max_val):
                if (x_index, y_index) in list(start_positions.keys()):
                    pos_info = CROSSWORD_GRID[start_positions[(x_index, y_index)]]
                    clue = start_positions[(x_index, y_index)]

                    guesses_ord = clues[clue]
                    all_guesses_constraint = list()

                    for guess_ord in guesses_ord:
                        guess_constraint = list()
                        x_i = x_index
                        y_i = y_index

                        for ch in guess_ord:
                            if isinstance(matrix[x_i][y_i], tuple):
                                # NOTE: _1 -> signifies the COLUMN index AND _2 -> signifies the ROW index
                                if pos_info["direction"] == "D":
                                    matrix_val = matrix[x_i][y_i][0]
                                elif pos_info["direction"] == "A":
                                    matrix_val = matrix[x_i][y_i][1]
                            else:
                                matrix_val = matrix[x_i][y_i]

                            guess_constraint.append(z3.And(matrix_val == ch))

                            if pos_info["direction"] == "D":
                                x_i += 1
                            elif pos_info["direction"] == "A":
                                y_i += 1

                        all_guesses_constraint.append(z3.And(guess_constraint))

                    clue_constraint.append(z3.Or(all_guesses_constraint))

        clues_constraint = z3.And(clue_constraint)

        return clues_constraint

    def apply_RC_constraint(self):
        clues = self.convert_clues_code()
        matrix, start_positions, max_val = self.get_grid()

        equality_constraint = list()

        for x_index in range(max_val):
            for y_index in range(max_val):
                if isinstance(matrix[x_index][y_index], tuple):
                    first = matrix[x_index][y_index][0]
                    second = matrix[x_index][y_index][1]

                    equality_constraint.append(z3.And(first == second))

        apply_RC_constraint = z3.And(equality_constraint)

        return apply_RC_constraint  

    def convert_clues_code(self):
        """converts all the clues (in string format) to a list of their ascii characters value
           example: "happy" -> [109, 117, 109, 98, 97, 105]
        """
        clues = self.get_hints()
        clues_ord = dict()

        for clue in clues:
            for guess in clues[clue]:
                try:
                    clues_ord[clue].append([ord(ch) for ch in guess.lower()])
                except:
                    clues_ord[clue] = [[ord(ch) for ch in guess.lower()]]

        return clues_ord    

    # Clues obtained from Graph DB
    def get_hints(self):
        hints = CROSSWORD_GRID_HINTS
        return hints

    def get_grid(self):
        clues = self.get_hints()
        start_positions = dict()

        # finding the square matrix size
        max_val = 0
        for clue in CROSSWORD_GRID:
            start_positions[CROSSWORD_GRID[clue]["start"]] = clue

            if CROSSWORD_GRID[clue]["start"][0] > max_val:
                max_val = CROSSWORD_GRID[clue]["start"][0]
            elif CROSSWORD_GRID[clue]["start"][1] > max_val:
                max_val = CROSSWORD_GRID[clue]["start"][1]

        # As the CROSSWORD_GRID starts from 0, we have to add 1 to get actual max_val
        max_val += 1

        matrix = [[None for index in range(max_val)] for index in range(max_val)]
        
        tabulate(matrix)

        # The following code encodes the matrix declared with z3.Int(VALUE)
        # If at a position value is: (z3.Int(VALUE)_1, z3.Int(VALUE)_2), it signifies intersection is taking place there
        # NOTE: _1 -> signifies the COLUMN index AND _2 -> signifies the ROW index
        for x_index in range(max_val):
            for y_index in range(max_val):

                if (x_index, y_index) in list(start_positions.keys()):
                    pos_info = CROSSWORD_GRID[start_positions[(x_index, y_index)]]
                    for i in range(pos_info["length"]):

                        if pos_info["direction"] == "A":
                            if isinstance(matrix[x_index][y_index + i], z3.z3.ArithRef):
                                matrix[x_index][y_index + i] = (z3.Int("alpha_" + str(x_index) + "_" + str(y_index + i) + "_1"), z3.Int("alpha_" + str(x_index) + "_" + str(y_index + i) + "_2"))
                            else:
                                matrix[x_index][y_index + i] = z3.Int("alpha_" + str(x_index) + "_" + str(y_index + i))

                        elif pos_info["direction"] == "D":
                            if isinstance(matrix[x_index + i][y_index], z3.z3.ArithRef):
                                matrix[x_index + i][y_index] = (z3.Int("alpha_" + str(x_index + i) + "_" + str(y_index) + "_1"), z3.Int("alpha_" + str(x_index + i) + "_" + str(y_index) + "_2"))
                            else:
                                matrix[x_index + i][y_index] = z3.Int("alpha_" + str(x_index + i) + "_" + str(y_index))

        return matrix, start_positions, max_val

    def solution(self):
        solved = self.apply_constraints()
        if solved is not None:
            solved_keys = [index.name() for index in solved]
            matrix, start_positions, max_val = self.get_grid()

            for x_index in range(max_val):
                for y_index in range(max_val):
                    if isinstance(matrix[x_index][y_index], tuple):
                        matrix_val = matrix[x_index][y_index][0]
                    elif matrix[x_index][y_index] != None:
                        matrix_val = matrix[x_index][y_index]
                    else:
                        continue

                    no = solved[matrix_val]
                    ch = chr(no.as_long())
                    matrix[x_index][y_index] = ch
        else:
            matrix = None

        return matrix    

In [65]:
c = Crossword()
c.solution()

[[None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None]]
[[None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None]]
unsat


In [66]:
x=[[None, None, None, None, None], 
[None, None, None, None, None], 
[None, None, None, None, None], 
[None, None, None, None, None], 
[None, None, None, None, None]]
x

[[None, None, None, None, None],
 [None, None, None, None, None],
 [None, None, None, None, None],
 [None, None, None, None, None],
 [None, None, None, None, None]]